# PYTHON PROJECT: STOCK PRICE PREDICTION USING LSTM 

## *This project demonstrates how to use Long Short-Term Memory (LSTM) neural networks to predict stock prices. The model predicts the next 7 days of stock prices for a given stock symbol. This project uses historical stock data from Yahoo Finance and is built using Python, TensorFlow, Keras, and Pandas.*

## 1. Import all the libraries needed

This part of the code imports various libraries and modules necessary for the stock price prediction project. Here's a breakdown of the imports:

1. `import numpy as np`: Imports the NumPy library, which is used for numerical operations on arrays and matrices.

2. `import pandas as pd`: Imports the pandas library, which is used for data manipulation and analysis.

3. `from datetime import datetime`: Imports the `datetime` class from the `datetime` module, which is used to handle dates and times.

4. `import yfinance as yf`: Imports the yfinance library, which is used to download financial data from Yahoo Finance.

5. `from sklearn.preprocessing import MinMaxScaler`: Imports the `MinMaxScaler` class from the scikit-learn library, which is used to scale the data between a specified range (e.g., 0 to 1).

6. `from sklearn.metrics import mean_squared_error`: Imports the mean_squared_error function from the scikit-learn library, which is used to calculate the mean squared error between the actual and predicted stock prices.

7. `from sklearn.model_selection import TimeSeriesSplit`: Imports the `TimeSeriesSplit` class from the scikit-learn library, which is used for time series cross-validation.

8. `from tensorflow.keras.models import Sequential`: Imports the Sequential class from the TensorFlow library, which is used to create a linear stack of layers for the neural network model.

9. `from tensorflow.keras.layers import Dense, LSTM, Dropout`: Imports the `Dense`, `LSTM`, and `Dropout` classes from the TensorFlow library, which are used to create the layers of the neural network model.

10. `from tensorflow.keras.optimizers import SGD, Adam`: Imports the `SGD` and `Adam` classes from the TensorFlow library, which are used as optimizers for the neural network model.

11. `from tensorflow.random import set_seed`: Imports the `set_seed` function from the TensorFlow library, which is used to set the random seed for reproducibility.

12. `from keras_tuner import RandomSearch`: Imports the `RandomSearch` class from the Keras Tuner library, which is used to perform random search for hyperparameter tuning.

13. `from pandas_datareader.data import DataReader`: Imports the `DataReader` function from the pandas-datareader library, which is used to fetch financial data from various sources, such as Yahoo Finance and Google Finance.

In [23]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.random import set_seed

from keras_tuner import RandomSearch

from pandas_datareader.data import DataReader

### 2. Download historical dataset with financial data about the stock prices using Yahoo Finance API 

1. `end = datetime.now()`: This line assigns the current date and time to the variable end. The `datetime.now()` function returns the present date and time as a datetime object.

2. `start = datetime(2016, end.month, end.day)`: This line creates a datetime object representing the start date for downloading the stock data. The year is set to 2016, while the month and day are set to the current month and day (retrieved from the end variable). This means the start date will be the same day and month as today, but in the year 2016.

3. `dataset = yf.download("AAPL", start, end)`: This line downloads the historical stock data for Apple Inc. (AAPL) between the start and end dates using the yfinance library. The downloaded data is stored in a DataFrame named dataset.

4. `tstart = 2016` and `tend = 2020`: These lines define two variables, tstart and tend, representing the years for splitting the dataset into a training set and a test set. The training set will contain data from 2016 to 2020, while the test set will contain data from 2021 onwards.


In [24]:
end = datetime.now()
start = datetime(2016, end.month, end.day) 
dataset = yf.download("AAPL", start, end) #You can input any stock symbol of your choice for analysis, for example, use "MSFT" for Microsoft stocks.
tstart = 2016
tend = 2020


[*********************100%***********************]  1 of 1 completed


### 3. Preprocess the data by splitting it into training and testing sets, scaling the data, and creating sequences for the LSTM model.

This part of the code is responsible for preparing the dataset for training and testing the model. Here's a step-by-step explanation of the code:

1. `train_test_plot(dataset, tstart, tend)`: This function takes in the dataset, a start year, and an end year, and plots the training and test data. The training data is plotted between the start and end years, while the test data is plotted from the end year onwards.

2. `train_test_split(dataset, tstart, tend)`: This function splits the dataset into training and test sets based on the given start and end years. The training set includes data between the start and end years, while the test set includes data from the end year onwards.

3. `train_test_split_values(dataset, tstart, tend)`: This function calls `train_test_split()` to get the training and test sets, and then returns their values as numpy arrays.

4. `training_set, test_set = train_test_split_values(dataset, tstart, tend)`: This line of code calls the `train_test_split_values()` function to obtain the training and test sets.

5. The MinMaxScaler is initialized with a feature range of (0, 1). The training set is then reshaped and scaled using this scaler. This scaling is done to ensure that all the values are between 0 and 1, which helps improve the performance of the neural network.

6. `split_sequence(sequence, window)`: This function takes a sequence of data and a window size as input, and creates input-output pairs for the sequence. The input is a sliding window of data, while the output is the next value in the sequence after the window.

7. `window_size` and `features`: These variables define the window size (60) and the number of features (1) for the input data.

8. `X_train, y_train = split_sequence(training_set_scaled, window_size)`: This line of code calls the split_sequence() function to create the input-output pairs for the training data.

9. `X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],features)`: This line reshapes the input training data (X_train) to the required 3-dimensional format for the LSTM model. The dimensions are (number of samples, time steps, number of features).

In [25]:
def train_test_plot(dataset, tstart, tend):
    dataset.loc[f"{tstart}":f"{tend}", "High"].plot(figsize=(16, 4), legend=True)
    dataset.loc[f"{tend+1}":, "High"].plot(figsize=(16, 4), legend=True)
    

def train_test_split(dataset, tstart, tend):
    train = dataset.loc[f"{tstart}":f"{tend}", "High"]
    test = dataset.loc[f"{tend+1}":, "High"]
    return train, test

def train_test_split_values(dataset, tstart, tend):
    train, test =  train_test_split(dataset, tstart, tend)
    return train.values, test.values

training_set, test_set = train_test_split_values(dataset, tstart, tend)


sc = MinMaxScaler(feature_range=(0, 1))
training_set = training_set.reshape(-1, 1)
training_set_scaled = sc.fit_transform(training_set)

def split_sequence(sequence, window):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + window
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

window_size = 60
features = 1

X_train, y_train = split_sequence(training_set_scaled, window_size)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],features)


### 4. Define the LSTM model, compile it, and fit it on the training data.

1. `model_lstm = Sequential()`: This line creates a sequential model, which is a linear stack of layers that can be built up one layer at a time.

2. The next few lines define the structure of the LSTM model:

- The first LSTM layer has 128 units, a "tanh" activation function, an input shape of (window_size, features), and return_sequences=True, which means it will return the full sequence of hidden states for each time step.
- A Dropout layer is added with a dropout rate of 0.2 to prevent overfitting.
- The second LSTM layer has 64 units and a "tanh" activation function.
- Another Dropout layer is added with a dropout rate of 0.1.
- A Dense layer with a single output unit is added to produce the predicted stock price.
3. `model_lstm.compile(optimizer=Adam(learning_rate=0.01), loss='mse')`: This line compiles the model with the Adam optimizer, a learning rate of 0.01, and mean squared error (mse) as the loss function.

4. `model_lstm.summary()` and `print(model_lstm)`: These lines print a summary of the model architecture.

5. `model_lstm.fit(X_train, y_train, epochs=30, batch_size=32)`: This line trains the LSTM model on the training data (X_train and y_train) for 30 epochs with a batch size of 32.

6. `dataset_total = dataset.loc[:,"High"]`: This line extracts the "High" price column from the dataset.

7. `inputs = dataset_total[len(dataset_total) - len(test_set) - window_size :].values`: This line creates an input array for the test set by taking the last `window_size` number of values from the dataset, combined with the test set values.

8. The next few lines reshape and scale the inputs using the MinMaxScaler (sc) that was previously fit on the training data.

9. `X_test, y_test = split_sequence(inputs, window_size)`: This line calls the split_sequence() function to create input-output pairs for the test data.

10. `X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], features)`: This line reshapes the input test data (X_test) to the required 3-dimensional format for the LSTM model.

11. `predicted_stock_price = model_lstm.predict(X_test)`: This line makes predictions on the test data using the trained LSTM model.

12. The next two lines inverse transform the predicted_stock_price and y_test using the MinMaxScaler (`sc`). This step converts the scaled values back to their original scale.

In [26]:
model_lstm = Sequential()
model_lstm.add(LSTM(units=128, activation="tanh", input_shape=(window_size, features), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(units=64, activation="tanh"))
model_lstm.add(Dropout(0.1))
model_lstm.add(Dense(units=1))

model_lstm.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

model_lstm.summary()
print(model_lstm)

model_lstm.fit(X_train, y_train, epochs=30, batch_size=32)

dataset_total = dataset.loc[:,"High"]
inputs = dataset_total[len(dataset_total) - len(test_set) - window_size :].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

X_test, y_test = split_sequence(inputs, window_size)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], features)
predicted_stock_price = model_lstm.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
y_test = sc.inverse_transform(y_test)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 60, 128)           66560     
                                                                 
 dropout_2 (Dropout)         (None, 60, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 116,033
Trainable params: 116,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
35/35 [==============================

### 5.Predict stock prices for the test set and calculate the root mean squared error (RMSE).

This part of the code defines a function called return_rmse that calculates the Root Mean Squared Error (RMSE) between the actual test data and the predicted data. RMSE is a commonly used metric to evaluate the performance of a regression model, such as a stock price prediction model in this case. Here's a breakdown of the code:

1. `def return_rmse(test, predicted)`:: This line defines the `return_rmse` function, which takes two arguments: `test` and `predicted`. `test` refers to the actual test data (y_test), and `predicted` refers to the predicted stock prices from the model (predicted_stock_price).

2. `rmse = np.sqrt(mean_squared_error(test, predicted))`: This line calculates the RMSE by first computing the mean squared error (MSE) between the test and predicted data using the `mean_squared_error()` function from the `sklearn.metrics` module. Then, it takes the square root of the MSE using NumPy's `np.sqrt()` function.

3. `print("The root mean squared error is {:.2f}.".format(rmse))`: This line prints the RMSE value with a message and formats the floating-point number with two decimal places using the `format()` method.

4. `return_rmse(y_test, predicted_stock_price)`: This line calls the `return_rmse` function with `y_test` and `predicted_stock_price` as arguments, which calculates and prints the RMSE for the stock price prediction model.

In [27]:
def return_rmse(test, predicted):
    rmse = np.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {:.2f}.".format(rmse))

return_rmse(y_test,predicted_stock_price)

The root mean squared error is 6.29.


### 6. Predict stock prices for the next 7 days and display the results.

This part of the code generates predictions for the stock prices for the next 7 days using the trained LSTM model. Here's a breakdown of the code:

1. `future_predictions = []`: This line initializes an empty list called `future_predictions` to store the predicted stock prices for the next 7 days.

2. `last_60_days = dataset_total[-window_size:]`.values: This line extracts the last 60 days of stock prices from the dataset, as it will be used as the input for predicting the next day's stock price.

3. `last_60_days = last_60_days.reshape(-1, 1)`: This line reshapes the `last_60_days` array to have one column and an appropriate number of rows.

4. `last_60_days = sc.transform(last_60_days)`: This line scales the `last_60_days` data using the same MinMaxScaler (`sc`) that was used for the training data.

5. `for i in range(7)`:: This line starts a loop that iterates 7 times, as we want to predict stock prices for the next 7 days.

6. `input_data = last_60_days[-window_size:].reshape(1, window_size, features)`: This line prepares the input data for the LSTM model by taking the last 60 days of the `last_60_days`array, reshaping it to match the input shape expected by the LSTM model (1, window_size, features).

7. `predicted_price = model_lstm.predict(input_data)`: This line makes a prediction for the next day's stock price using the LSTM model.

8. `last_60_days = np.concatenate((last_60_days, predicted_price), axis=0)`: This line appends the predicted stock price to the `last_60_days` array, which will be used as input for the next prediction. It also removes the oldest stock price from the array.

9. `predicted_price = sc.inverse_transform(predicted_price)`: This line inverse transforms the predicted stock price to its original scale using the MinMaxScaler (`sc`).

10. `future_predictions.append(predicted_price[0][0])`: This line adds the predicted stock price to the future_predictions list.

11. `print("Predicted stock prices for the next 7 days:")`: This line prints a message to indicate that the predicted stock prices for the next 7 days will be displayed.

12. `for i, price in enumerate(future_predictions)`:: This line starts a loop that iterates through the `future_predictions` list.

13. `print(f"Day {i + 1}: {price:.2f}")`: This line prints the predicted stock price for each day in a formatted string with two decimal places.

In [28]:
future_predictions = []

last_60_days = dataset_total[-window_size:].values
last_60_days = last_60_days.reshape(-1, 1)
last_60_days = sc.transform(last_60_days)

for i in range(7):
    input_data = last_60_days[-window_size:].reshape(1, window_size, features)
    predicted_price = model_lstm.predict(input_data)
    last_60_days = np.concatenate((last_60_days, predicted_price), axis=0)
    predicted_price = sc.inverse_transform(predicted_price)
    future_predictions.append(predicted_price[0][0])

print("Predicted stock prices for the next 7 days:")
for i, price in enumerate(future_predictions):
    print(f"Day {i + 1}: {price:.2f}")

1/1 [==============================] - 0s 29ms/step
Predicted stock prices for the next 7 days:
Day 1: 164.65
Day 2: 163.06
Day 3: 160.94
Day 4: 158.53
Day 5: 156.06
Day 6: 153.74
Day 7: 151.58


### 7. Tune the parameters of the model

This part of the code defines a function called build_model that takes a hp (hyperparameter) argument and constructs an LSTM model using the Keras library. It also sets up a random search to find the optimal hyperparameters for the model using the Keras Tuner library. Here's a breakdown of the code:

1. `def build_model(hp):`: This line defines the build_model function, which accepts a hp argument representing the hyperparameters of the model.

2. `model = Sequential()`: This line creates an empty Sequential model.

3. The next few lines add two LSTM layers with dropout layers in between:

a. `model.add(LSTM(...))`: These lines add LSTM layers with the specified number of units, activation function, and input shape. The number of units is determined by the hyperparameter optimization process, with a range between 32 and 256 and a step of 32.

b. `model.add(Dropout(...))`: These lines add dropout layers to the model to prevent overfitting. The dropout rate is also determined by the hyperparameter optimization process, with a range between 0.0 and 0.5 and a step of 0.1.

4. `model.add(Dense(units=1))`: This line adds a Dense layer with a single output unit, which represents the predicted stock price.

5. `model.compile(...)`: This line compiles the model, specifying the optimizer (Adam), learning rate, and loss function (mean squared error). The learning rate is also determined by the hyperparameter optimization process, with a choice among 0.01, 0.001, and 0.0001.

6. `return model`: This line returns the constructed model.

7. `tuner = RandomSearch(...`): This line creates a `RandomSearch` object, which searches for the best hyperparameters for the model. It takes the `build_model` function as input, along with the objective (minimizing validation loss), the number of trials, the number of executions per trial, and the directory and project name to store the results.

8. `tuner.search_space_summary()`: This line prints a summary of the search space for the hyperparameters.

9. `tuner.search(...)`: This line runs the random search for the best hyperparameters, training the model with the specified number of epochs, batch size, validation split, and verbosity level.

10. `tuner.results_summary()`: This line prints a summary of the best hyperparameters found during the search.

11. `best_model = tuner.get_best_models(num_models=1)[0]`: This line retrieves the best model found during the search and assigns it to the `best_model` variable.

In [ ]:
def build_model(hp):
    model = Sequential()

    model.add(LSTM(units=hp.Int('units_1', min_value=32, max_value=256, step=32),
                   activation='tanh',
                   input_shape=(window_size, features),
                   return_sequences=True))
    
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(LSTM(units=hp.Int('units_2', min_value=32, max_value=256, step=32),
                   activation='tanh'))
    
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(units=1))

    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='mse')

    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=30,
    executions_per_trial=3,
    directory='my_dir',
    project_name='stock_prediction')

tuner.search_space_summary()

tuner.search(X_train, y_train,
             epochs=50,
             batch_size=32,
             validation_split=0.2,
             verbose=1)

tuner.results_summary()

best_model = tuner.get_best_models(num_models=1)[0]